In [62]:
import json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
scan = helpers.scan

es = Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [56]:
results = scan(es, query = 
{
  "query": {
    "bool": {
      "must": [
        {
          "exists": {
            "field": "bm.overall"
          }
        }
      ]
    }
  }
}, index = 'redditors')

all_keys = set()
summary = {}
count = 0
for result in results:
    count = count + 1
    if count % 5000 == 0:
        print(count)
    
    doc = result['_source']
    mbti = doc['bm']['overall']
    
    for subreddit in doc['subreddits']:
        if subreddit not in summary.keys():
            summary[subreddit] = {
                "count": 0
            }
            
        summary[subreddit]['count'] = summary[subreddit]['count'] + 1
    
        for key in mbti.keys():
            if key == 'type':
                key = mbti['type']
            
            all_keys.add(key)
            if key in summary[subreddit]:
                summary[subreddit][key]['count'] = summary[subreddit][key]['count'] + 1
            else:
                summary[subreddit][key] = { "count": 0 }

5000
10000
15000
20000


In [57]:
all_keys

{'EIFJ',
 'EIFP',
 'EITJ',
 'EITP',
 'ESFJ',
 'ESFP',
 'ESTJ',
 'ESTP',
 'Extrovert',
 'Feeling',
 'IIFJ',
 'IIFP',
 'IITJ',
 'IITP',
 'ISFJ',
 'ISFP',
 'ISTJ',
 'ISTP',
 'Introvert',
 'Intuitive',
 'Judging',
 'Perceiving',
 'Sensing',
 'Thinking'}

In [60]:
for name in summary:
    subreddit = summary[name]
    
    for key in subreddit:
        if isinstance(subreddit[key], dict):
            subreddit[key]["percent"] = (subreddit[key]["count"] / subreddit["count"]) * 100.0
    
    for key in all_keys:
        if key not in subreddit.keys():
            subreddit[key] = { "count": 0, "percent": 0.0}

In [70]:
l = list()

for name in summary:
    summary[name]['name'] = name
    l.append(summary[name])
    
with open('subreddit-summary.json', 'w') as fp:
    json.dump(l, fp)

In [69]:
l[0]

{'count': 1965,
 'Extrovert': {'count': 954, 'percent': 48.54961832061069},
 'Feeling': {'count': 225, 'percent': 11.450381679389313},
 'Intuitive': {'count': 733, 'percent': 37.30279898218829},
 'Perceiving': {'count': 1705, 'percent': 86.76844783715013},
 'EIFP': {'count': 38, 'percent': 1.9338422391857506},
 'Sensing': {'count': 1230, 'percent': 62.59541984732825},
 'ISFP': {'count': 33, 'percent': 1.6793893129770994},
 'Introvert': {'count': 1009, 'percent': 51.348600508905854},
 'IIFP': {'count': 56, 'percent': 2.849872773536896},
 'Thinking': {'count': 1738, 'percent': 88.44783715012723},
 'ESTP': {'count': 634, 'percent': 32.264631043257},
 'EITP': {'count': 185, 'percent': 9.414758269720101},
 'ISTP': {'count': 419, 'percent': 21.323155216284988},
 'Judging': {'count': 258, 'percent': 13.129770992366414},
 'IITJ': {'count': 105, 'percent': 5.343511450381679},
 'IITP': {'count': 291, 'percent': 14.80916030534351},
 'ISTJ': {'count': 58, 'percent': 2.9516539440203564},
 'ESFP': {